In [1]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from jira_api import JiraApiWrapper
from jira_toolkit import JiraToolkit

from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["JIRA_USERNAME"] = os.getenv('JIRA_USERNAME')
os.environ["JIRA_API_TOKEN"] = os.getenv('JIRA_API_TOKEN')
os.environ["JIRA_INSTANCE_URL"] = os.getenv('JIRA_INSTANCE_URL')

llm = OpenAI(temperature=0)

jira = JiraApiWrapper()
jira_toolkit = JiraToolkit.from_jira_api_wrapper(jira)

tools = jira_toolkit.get_tools()

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True)

In [2]:
agent.run("make a new issue for project OP to remind me to make more fried rice")



> Entering new AgentExecutor chain...


Creating issue "Make more fried rice"


 I need to create an issue
Action: Create Issue
Action Input: {"summary": "Make more fried rice", "description": "Reminder to make more fried rice", "project": { "key": "OP" }, "issuetype": {"name": "Task"}, "priority": {"name": "Low"}}
Observation: {'id': '10006', 'key': 'OP-7', 'self': 'https://thinkovator.atlassian.net/rest/api/2/issue/10006'}
Thought: I now know the final answer
Final Answer: A new issue with the summary "Make more fried rice" has been created in project OP.

> Finished chain.


'A new issue with the summary "Make more fried rice" has been created in project OP.'

In [4]:
import requests
from requests.auth import HTTPBasicAuth
import json

base_url = os.getenv('JIRA_INSTANCE_URL')
create_url = f"{base_url}/rest/api/2/issue"

auth = HTTPBasicAuth(os.getenv('JIRA_USERNAME'), os.getenv('JIRA_API_TOKEN'))

metadata_url = f"{base_url}/rest/api/2/issue/createmeta"

headers = {
  "Accept": "application/json"
}

response = requests.request(
   "GET",
   metadata_url,
   headers=headers,
   auth=auth
)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))


{
    "expand": "projects",
    "projects": [
        {
            "avatarUrls": {
                "16x16": "https://thinkovator.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10411?size=xsmall",
                "24x24": "https://thinkovator.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10411?size=small",
                "32x32": "https://thinkovator.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10411?size=medium",
                "48x48": "https://thinkovator.atlassian.net/rest/api/2/universal_avatar/view/type/project/avatar/10411"
            },
            "id": "10000",
            "issuetypes": [
                {
                    "description": "Tasks track small, distinct pieces of work.",
                    "iconUrl": "https://thinkovator.atlassian.net/rest/api/2/universal_avatar/view/type/issuetype/avatar/10318?size=medium",
                    "id": "10001",
                    "name": "Task",
                

In [5]:

payload = json.dumps( {
  "fields": {
    "issuetype": {
      "name": "Task"
    },
    "priority": {
      "name": "Low"
    },
    "project": {
      "key": "OP"
    },
    "summary": "Main order flow broken",
    "description": "What do you think"
  }
} )

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json"
}

response = requests.request(
   "POST",
   create_url,
   data=payload,
   headers=headers,
   auth=auth
)

print(response.status_code)
print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

201
{
    "id": "10005",
    "key": "OP-6",
    "self": "https://thinkovator.atlassian.net/rest/api/2/issue/10005"
}
